In [1]:
import os

In [2]:

os.chdir("../")

In [3]:
pwd

'C:\\Users\\youss\\OneDrive\\Bureau\\computer vision\\Projet_sign recognitin\\keypointsapproach\\classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    classes:int 

In [5]:
from classification.constants import *
from classification.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"Data")
        create_directories([
            Path(training.root_dir)
        ])


        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            classes=params.CLASSES
        )
        return training_config

In [11]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from classification import logger
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import mlflow
import mlflow.keras
#Component 
class training_Model:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )


    def Processing_Data(self):
        Train_Data=os.path.join(self.config.training_data,'Train')
        X_train=[]
        y_train=[]
        for classe in os.listdir(Train_Data):
              
          for img in os.listdir(os.path.join(Train_Data,classe)) :
            img_path=os.path.join(Train_Data,classe,img)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(75,75))
            x = tf.keras.preprocessing.image.img_to_array(img)

            x = tf.keras.applications.vgg16.preprocess_input(x)
            X_train.append(x)
            y_train.append(classe)
        

      
  
        val_Data=os.path.join(self.config.training_data,'valid')
        X_val=[]
        y_val=[]

        for classe in os.listdir(val_Data):
            
          for img in os.listdir(os.path.join(val_Data,classe)) :
            img_path=os.path.join(val_Data,classe,img)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(75,75))
            x = tf.keras.preprocessing.image.img_to_array(img)

            x = tf.keras.applications.vgg16.preprocess_input(x)
            X_val.append(x)
            y_val.append(classe)


                # Convert y_train to a NumPy array
        y_train_array = np.array(y_train)
        y_val_array = np.array(y_val)

        # Reshape y_train for one-hot encoding
        y_train_reshaped = y_train_array.reshape(-1, 1)
        y_val_reshaped = y_val_array.reshape(-1, 1)

        # Initialize the one-hot encoder
        onehot_encoder = OneHotEncoder(sparse=False)
      
        # Fit one-hot encoder and transform y_train
        y_train_onehot = onehot_encoder.fit_transform(y_train_reshaped)
        y_val_onehot = onehot_encoder.transform(y_val_reshaped)

        self.hotencoder=  onehot_encoder


        
        X_train=np.array(X_train)
        X_val=np.array(X_val)
        self.X_train=X_train
        self.y_train=  y_train_onehot
        self.X_val=X_val
        self.y_val=   y_val_onehot
      
        

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        
        early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

        self.model.fit(
            self.X_train,self.y_train,
            epochs=self.config.params_epochs,
            validation_data=(self.X_val,self.y_val),
            batch_size=self.config.params_batch_size,callbacks=[early_stopping_callback]
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        full_model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(75, 75, 3), padding = "same"),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(4, activation='softmax')
        ])
                    
        # Define optimizer with learning rate decay
        batches_per_epoch = len(self.X_train)*self.config.params_batch_size
        lr_decay = (1./0.75 - 1) / batches_per_epoch
        opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001, decay=lr_decay)
        
        # Compile the model with Adam optimizer
        full_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [13]:
try:
    config = ConfigurationManager()
    Training_model_config = config.get_training_config()
    Training_model = training_Model(config=Training_model_config)
    Training_model.get_base_model()
    
    Training_model.Processing_Data()
    Training_model.train()
except Exception as e:
    raise e

[2024-03-27 02:58:13,561: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-27 02:58:13,565: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-27 02:58:13,567: INFO: common: created directory at: artifacts]
[2024-03-27 02:58:13,567: INFO: common: created directory at: artifacts\training]
Epoch 1/50


C:\Users\youss\anaconda3\envs\classification_cancer\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ValueError: in user code:

    File "C:\Users\youss\anaconda3\envs\classification_cancer\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\youss\anaconda3\envs\classification_cancer\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\youss\anaconda3\envs\classification_cancer\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\youss\anaconda3\envs\classification_cancer\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\youss\anaconda3\envs\classification_cancer\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\youss\anaconda3\env

In [107]:
def Processing_Data():
        Train_Data=os.path.join("artifacts/data_ingestion/Data",'Train')
        X_train=[]
        y_train=[]
        for classe in os.listdir(Train_Data):
              
          for img in os.listdir(os.path.join(Train_Data,classe)) :
            img_path=os.path.join(Train_Data,classe,img)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(75,75)) 
            x = tf.keras.preprocessing.image.img_to_array(img) / 255.0

            #x = tf.keras.applications.vgg16.preprocess_input(x)
            X_train.append(x)
            y_train.append(classe)
        

      
  
        val_Data=os.path.join("artifacts/data_ingestion/Data",'valid')
        X_val=[]
        y_val=[]

        for classe in os.listdir(val_Data):
            
          for img in os.listdir(os.path.join(val_Data,classe)) :
            img_path=os.path.join(val_Data,classe,img)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(75,75)) 
            x = tf.keras.preprocessing.image.img_to_array(img) / 255.0

            #x = tf.keras.applications.vgg16.preprocess_input(x)
            X_val.append(x)
            y_val.append(classe)


                # Convert y_train to a NumPy array
        y_train_array = np.array(y_train)
        y_val_array = np.array(y_val)

        # Reshape y_train for one-hot encoding
        y_train_reshaped = y_train_array.reshape(-1, 1)
        y_val_reshaped = y_val_array.reshape(-1, 1)

        # Initialize the one-hot encoder
        onehot_encoder = OneHotEncoder(sparse=False)
      
        # Fit one-hot encoder and transform y_train
        y_train_onehot = onehot_encoder.fit_transform(y_train_reshaped)
        y_val_onehot = onehot_encoder.transform(y_val_reshaped)

        


        
        X_train=np.array(X_train)
        X_val=np.array(X_val)
        
        y_train=  y_train_onehot
      
        y_val=   y_val_onehot
        return X_train,y_train,X_val,y_val
      
        

In [113]:

    
    def train(model, X_train,y_train,X_val,y_val):
        

        model.fit(
            X_train,y_train,
            epochs=100,
            validation_data=(X_val,y_val),
            batch_size=24
        )




    def _prepare_full_model(X_train):
    

        full_model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(75, 75, 3), padding = "same"),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(4, activation='softmax')
        ])
                    
        # Define optimizer with learning rate decay
        batches_per_epoch = len(X_train)*24
        lr_decay = (1./0.75 - 1) / batches_per_epoch
        opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001, decay=lr_decay)
        
        # Compile the model with Adam optimizer
        full_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        

        full_model.summary()
        return full_model

In [109]:
 X_train,y_train,X_val,y_val=Processing_Data()

C:\Users\youss\anaconda3\envs\classification_cancer\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [114]:
model=_prepare_full_model(X_train)


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 75, 75, 32)        896       
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 37, 37, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_33 (Conv2D)          (None, 35, 35, 64)        18496     
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 17, 17, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_34 (Conv2D)          (None, 15, 15, 128)       73856     
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 7, 7, 128)      

In [115]:
train(model,X_train,y_train,X_val,y_val)

Epoch 1/100
26/26 [==============================] - 4s 105ms/step - loss: 1.3835 - accuracy: 0.2741 - val_loss: 1.3818 - val_accuracy: 0.3333
Epoch 2/100
26/26 [==============================] - 2s 94ms/step - loss: 1.3661 - accuracy: 0.3491 - val_loss: 1.3757 - val_accuracy: 0.1806
Epoch 3/100
26/26 [==============================] - 3s 97ms/step - loss: 1.3292 - accuracy: 0.3442 - val_loss: 1.3551 - val_accuracy: 0.3889
Epoch 4/100
26/26 [==============================] - 2s 94ms/step - loss: 1.3079 - accuracy: 0.3736 - val_loss: 1.3150 - val_accuracy: 0.4028
Epoch 5/100
26/26 [==============================] - 2s 92ms/step - loss: 1.2508 - accuracy: 0.4307 - val_loss: 1.2597 - val_accuracy: 0.4583
Epoch 6/100
26/26 [==============================] - 2s 94ms/step - loss: 1.2366 - accuracy: 0.4372 - val_loss: 1.2139 - val_accuracy: 0.4583
Epoch 7/100
26/26 [==============================] - 2s 93ms/step - loss: 1.1772 - accuracy: 0.4731 - val_loss: 1.1745 - val_accuracy: 0.4722
Epoch